In [1]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
from src.generator import RAGKGGenerator
from src.llms import OpenAIModel, HuggingFaceLLM
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('cs2')
token = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN') 
os.environ['OPENAI_API_KEY'] = token

cs2_chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/


In [ ]:
# data = pd.read_csv('data/sorting.csv')
# data.columns = ['concept', 'outcome']
data = pd.read_csv('data/dsa_clifford_a_shaffer_3_2_java.csv')
data

In [ ]:
actual_concepts = data['concepts'][3].split('::')
actual_concepts

In [ ]:
extractor = RAGKGGenerator(
        [cs2_chapters[3]],
        300,
        100,
        OpenAIModel(),
        link,
)

In [ ]:
from src.metrics import SemanticSimilarity, Contextual_F1, AnswerCorrectness
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric, ContextualRelevancyMetric


metrics = [AnswerRelevancyMetric(model = extractor.llm), AnswerCorrectness(model = extractor.llm), FaithfulnessMetric(model = extractor.llm), ContextualPrecisionMetric(model = extractor.llm), ContextualRecallMetric(model = extractor.llm), SemanticSimilarity(st_model = extractor.embedding_model)]

<p>this cell takes a while to run so i commented it out for now</p>

In [ ]:
# for i in range(5):
extractor.identify_concepts(5) 
result = extractor.evaluate('concepts', 5, actual_concepts, metrics)
# print('-' * 20)
# print(f'OPENAI MODEL: {extractor.llm.get_model_name()}')
# print(f'TEMPERATURE: {extractor.llm.temperature}')
# print(f'SENTENCE TRANSFORMER: {extractor.embedding_model}')
# print(f'TEXTBOOK: dsa_2214')
# print(f'CHAPTERS TESTED: {extractor.chapters}')
# print('-' * 20)
# for i in result:
#     print(f"METRIC: {i['name']} ----> SCORE: {i['score']}")
#     print()
#     print(f"REASON: {i['reason']}")
#     print() 
#     print(f"QUERY: {i['input']}")
#     print()
#     print(f"OUTPUT: {i['output']}")
#     print('-' * 20)

In [ ]:
result

In [ ]:
full_extractor = RAGKGGenerator(
    cs2_chapters, 
    500, 
    100, 
    OpenAIModel(),
    link
)

In [ ]:
full_extractor.identify_concepts(10) # concepts from cs2 textbook 

In [ ]:
full_extractor.identify_outcomes(10) # outcomes from textbook

In [ ]:
full_extractor.identify_dependencies('concepts') # extracting chapter dependencies using generated concepts 

In [ ]:
cs2_kg = full_extractor.build_knowledge_graph(full_extractor.dependencies, full_extractor.concepts, full_extractor.outcomes)

In [ ]:
full_extractor.draw_knowledge_graph(cs2_kg, './visualizations/cs2_kg.html') # saves kg to cs2_kg.html

In [ ]:
clusters = full_extractor.build_terminology(build_using = [concept for l in full_extractor.concepts for concept in l])
# need list[str] to create clusters
# creating clusters using concepts, other options include main topics and outcomes 

In [ ]:
full_extractor.visualize_hierarchy([clusters], visual_type = 'cluster map')

In [ ]:
full_extractor.terminology

In [2]:
# from src.retrieval import RetrievalSystem
from src.retrievers import TransformerRetriever, VectorDBRetriever
# tr = TransformerRetriever(link, 500, 100, 'msmarco-distilbert-base-tas-b')
vdbr = VectorDBRetriever(link, 500, 100, 'msmarco-distilbert-base-tas-b')

In [ ]:
tr.pipeline('Lists, Stacks, and Queues', OpenAIModel()) 

In [3]:
vdbr.pipeline('What are queues?', OpenAIModel())

['one of the most important aspects of a course in data structures is that it is where students really learn to program using pointers and dynamic memory al - location, by implementing data structures such as linked lists and trees. it is often where students truly learn recursion. in our curriculum, this is the ﬁrst course where students do signiﬁcant design, because it often requires real data structures to mo - tivate signiﬁcant design exercises. finally, the fundamental differences between memory - based and disk - based data access cannot be appreciated without practical programming experience. for all of these reasons, a data structures course cannot succeed without a signiﬁcant programming component. in our department, the data structures course is one of the most difﬁcult programming course in the curriculum. students should also work problems to develop their analytical abilities. i pro - vide over 450 exercises and suggestions for programming projects. i urge readers to take 

In [ ]:
vdbr.semantic_search('What are queues?')

In [ ]:
hadoop_chapters = [
    'Meet Hadoop',
    'MapReduce',
    'The Hadoop Distributed Filesystem',
    'YARN',
    'Hadoop I/O',
    'Developing a MapReduce Application',
    'How MapReduce Works',
    'MapReduce Types and Formats',
    'MapReduce Features',
    'Setting up a Hadoop Cluster',
    'Administering Hadoop',
    'Avro',
    'Parquet',
    'Flume',
    'Sqoop',
    'Pig',
    'Hive',
    'Crunch',
    'Spark',
    'HBase',
    'Zookeeper',
    'Composable Data at Cerner',
    'Biological Data Science: Saving Lives with Software.',
    'Cascading'
]

hadoop_extractor = RAGKGGenerator(
    hadoop_chapters,
    500,
    100,
    OpenAIModel(),
    os.getenv('ccda'),
    'data/Syllabus for 202380-Fall 2023-ITCS-3190-001-Cloud Comp for Data Analysis.pdf'
)

In [ ]:
# clusters, is_a_relationships, topic_objectives, syllabus_objectives, main_topics = hadoop_extractor.syllabus_pipline()
kg = hadoop_extractor.syllabus_pipeline()

In [ ]:
hadoop_extractor.clusters

In [ ]:
hadoop_extractor.draw_knowledge_graph(kg, './visualizations/syllabus_hadoop_kg.html')

In [ ]:
hadoop_extractor.visualize_hierarchy(hadoop_extractor.clusters, 'cluster map')

In [ ]:
from src.metrics import AnswerCorrectness
from deepeval.test_case import LLMTestCase
cloud_computing_learning_topics = ['Distributed Computing and Clouds', 'Data Analysis Algorithms', 'Hadoop', 'HDFS', 'YARN', 'MapReduce', 'Pig', 'Hive', 'Spark', 'Information Retrieval', 'Page Rank', 'Web Search']
import warnings
warnings.filterwarnings('ignore')

ac = AnswerCorrectness()
ac.measure(LLMTestCase('No input given.', ' '.join(hadoop_extractor.main_topics), ' '.join(cloud_computing_learning_topics)))

print('actual:', hadoop_extractor.main_topics)
print('expected:', cloud_computing_learning_topics)
print(ac.score)
print(ac.reason)

In [ ]:
topics = hadoop_extractor.identify_main_topics()
topics

In [ ]:
hadoop_extractor.objectives_from_syllabus()

In [ ]:
relations = hadoop_extractor.identify_main_topic_relations()

In [ ]:
hadoop_extractor.draw_graph(relations)

## 